In [4]:
import pandas as pd
import datetime
import gzip
import io
import os
from nltk.sentiment import SentimentIntensityAnalyzer

---

## <center>Funciones</center>

In [109]:
def extraer_fecha(time):
    time_seconds = time / 1000  # Convertir milisegundos a segundos
    fecha_hora = datetime.datetime.utcfromtimestamp(time_seconds)

    # Eliminar los milisegundos
    fecha_hora = fecha_hora.replace(microsecond=0)

    return fecha_hora


In [110]:
def extraer_fecha_opinion(time):
    
    time = int(time)
    
    time_seconds = time / 1000  # Convertir milisegundos a segundos
    
    fecha_hora = datetime.datetime.utcfromtimestamp(time_seconds)

    # Eliminar los milisegundos
    fecha_hora = fecha_hora.replace(microsecond=0)

    return fecha_hora

---

In [140]:
dicc = {}

for x in range(1,12):
    dicc[x] = pd.read_json(f'datasets\\Google Maps\\reviews-estados\\review-Alabama\\{x}.json', lines=True)
    
    
# Lista para almacenar los DataFrames
lista_dataframes = []

# Iterar sobre los valores del diccionario (los DataFrames)
for df in dicc.values():
    lista_dataframes.append(df)

# Concatenar los DataFrames en uno solo
json = pd.concat(lista_dataframes, ignore_index=True)


In [141]:
''' Elimino la columna pics, las imagenes son muy chicas y no tienen relevancia para su proximo analisis, en cualquier caso si es necesario acceder a las mismas, una alternativa mucho mejor es utilizar la api o scraping con su id'''
json.drop(columns=['pics'],inplace=True) 

In [142]:
''' Utilizo la funcion creada anteriormente, para convertir los milisegundos a una fecha entendible, luego lo separo en 2 columnas para poder comparar el tiempo de respuesta con el de la opinion propiamente dicha'''
json['time'] = json['time'].apply(extraer_fecha)

In [143]:
'''

Desanido la columna resp, para luego poder realizar calculos con el tiempo de respuesta y de ser necesario el analisis de sentimiento de la respuesta por separado.

'''
json['fecha_respuesta'] = None
json['respuesta'] = None

# Desanidar la columna 'resp' si no es None y asignar valores a las nuevas columnas
json.loc[json['resp'].notnull(), 'fecha_respuesta'] = json.loc[json['resp'].notnull(), 'resp'].apply(lambda x: x.get('time') if x else None)
json.loc[json['resp'].notnull(), 'respuesta'] = json.loc[json['resp'].notnull(), 'resp'].apply(lambda x: x.get('text') if x else None)

# Convertir la columna 'respuesta' a tipo de datos 'object' para permitir listas o strings
json['respuesta'] = json['respuesta'].astype('object')

# Luego, realizar la asignación de valores a la columna 'respuesta'
json.loc[json['resp'].notnull(), 'respuesta'] = json.loc[json['resp'].notnull(), 'resp'].apply(lambda x: x.get('text') if x else None)

json.drop(columns=['resp'],inplace=True)

In [144]:
json['fecha_respuesta'].fillna(json['time'], inplace=True)

In [145]:
json['fecha_respuesta'] = json['fecha_respuesta'].astype(str)

In [146]:
# Aplicar la función 'extraer_anio()' a las filas que cumplen con la condición de longitud
json.loc[json['fecha_respuesta'].str.len() < 14, 'fecha_respuesta'] = json.loc[json['fecha_respuesta'].str.len() < 14, 'fecha_respuesta'].apply(extraer_fecha_opinion)
json['fecha_respuesta'] = pd.to_datetime(json['fecha_respuesta'], format='%Y-%m-%d %H:%M:%S')

In [147]:
json2 = json.copy()

json2['anio'] = json['fecha_respuesta'].dt.year  # Extraer el año y crear una nueva columna 'anio'

resultados = json2.groupby('anio').size()  # Agrupar por año y contar la cantidad de registros

resultados

anio
2008         8
2009        12
2010       133
2011       524
2012       770
2013      1346
2014      1729
2015      6117
2016     42198
2017    198758
2018    385741
2019    513182
2020    322927
2021    176555
dtype: int64

In [13]:
json

,gmap_id,latitude,longitude,category,avg_rating
0,0x88f16e41928ff687:0x883dad4fd048e8f8,32.388300,-83.357100,[Pharmacy],4.9
1,0x80c2c98c0e3c16fd:0x29ec8a728764fdf9,34.018891,-118.215290,[Textile exporter],4.5
2,0x80c2c778e3b73d33:0xbdc58662a4a97d49,34.058092,-118.292130,[Korean restaurant],4.4
3,0x80c2c89923b27a41:0x32041559418d447,34.023669,-118.232930,[Fabric store],3.3
4,0x80c2c632f933b073:0xc31785961fe826a6,34.036694,-118.249421,[Fabric store],4.3
...,...,...,...,...,...
3025006,0x89c6c74f43a49b55:0x6be6995921c58b12,39.947254,-75.164953,"[Steak house, Fine dining restaurant, Seafood ...",4.6
3025007,0x89da1787d3fdc4a5:0xce92ea6e3cd4d1cc,42.788636,-76.122120,[Chevrolet dealer],4.4
3025008,0x89da6b9ce832726f:0xbd0d3630f1e3b42e,42.603831,-76.165286,[Veterinarian],4.3
3025009,0x88d9e872a9e39f3d:0x5de29f50507e5d73,25.491641,-80.456076,[Animal feed store],4.9
